In [1]:
import os
import sys
import pprint

base_dir = 'F:\Thesis_ssd\MasterThesis3.0'
os.chdir(base_dir)

from Classes.DataProcessing.LoadData import LoadData
from Classes.DataProcessing.DataHandler import DataHandler
from Classes.DataProcessing.DataGenerator import DataGenerator
from Classes.Modeling.GridSearchResultProcessor import GridSearchResultProcessor
from Classes.Modeling.CustomCallback import CustomCallback
from Classes.Scaling.ScalerFitter import ScalerFitter
from Classes.Scaling.MinMaxScalerFitter import MinMaxScalerFitter
from Classes.Scaling.StandardScalerFitter import StandardScalerFitter
from Classes.Modeling.ResultFitter import ResultFitter
import json

In [16]:
load_args = {
    'earth_explo_only' : True,
    'noise_earth_only' : False,
    'downsample' : True,
    'upsample' : True,
    'frac_diff' : 0.5,
    'seed' : 1,
    'subsample_size' : 0.25
}

loadData = LoadData(**load_args)

full_ds, train_ds, val_ds, test_ds = loadData.get_datasets()
noise_ds = loadData.noise_ds
handler = DataHandler(loadData)
dataGen = DataGenerator(loadData)
resultFitter = ResultFitter(loadData)

AttributeError: 'LoadData' object has no attribute 'getDatasets'

In [6]:
result_file_name = 'results_6_noiseNotNoise_timeAug_sscale_noiseAug_earlyS.csv'
model = resultFitter.fit_from_csv_and_index(result_file_name, 0, 2, 10, use_tensorboard = False, 
                               use_liveplots = True, use_custom_callback = False)

NameError: name 'resultFitter' is not defined

In [2]:
result_file_name = 'results_LSTM_NARROW_noiseNotNoise_timeAug_mmscale_noiseAug_earlyS.csv'
df = GridSearchResultProcessor().get_results_df_by_name(result_file_name, 2)

In [5]:
df[df.columns[:15]]

,batch_size,epochs,learning_rate,num_layers,optimizer,activation,decay_sequence,dropout_rate,filters,kernel_size,l1_r,l2_r,output_layer_activation,padding,start_neurons
0,64,33,0.01,2,sgd,tanh,"[1, 2]",0.3,17,3,0.0001,0.001,sigmoid,same,32


In [4]:
df[df.columns[13:]]

,padding,start_neurons,train_loss,train_accuracy,train_precision,train_recall,val_loss,val_accuracy,val_precision,val_recall
0,same,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
fit_from_csv_and_index(result_file_name, 0, 2)

NameError: name 'fit_from_csv_and_index' is not defined

In [ ]:
li = fit_from_csv_and_index(result_file_name, 0, 2)

In [ ]:
mysplit(result_fi)

In [60]:
class A(B):
    def __init__(self, hey):
        self.hey = hey

class B:
    def __init__(self):
        pass
    def check(self):
        print(hasattr(self, 'hey'))

In [61]:
A = A("bye")

In [62]:
A.check()

AttributeError: 'A' object has no attribute 'check'

In [58]:
hasattr(A, 'hey')

True